
<center><img src="https://www.usergioarboleda.edu.co/wp-content/uploads/ultimatum/imagens/logo-mobile-UniversidadSergioArboleda.png">
<h1>Universidad Sergio Arboleda</h1>
<h3>Ciencias de la Computacion e IA
<h4>Metricas de Rendimiento</h4>
<p><b>INTEGRANTES:</b></p>
<p>Daniel Alejandro Olarte Ávila</p>
<p>22 de Noviembre 2022</p>
<p>Profesor: John Corredor</p></center>

<h1> IMPORTANTE: Debido a el gran tamaño de datos que tienen los datasets no se pudo subir dichos datasets al repositorio personal, ni comprimidos, por lo cual se enviaron los datasets por aparte para poder evidenciar todo el trabajo en el caso que se desee volver a ejecutar y/o editar el cuaderno

# IMPORTACIONES E INSTALACIONES

In [ ]:
#Modulo para realizar mapas el cual se va a trabajar para definir en que lugar hubo delitos segun el dataset
#pip install folium

In [2]:
#Se importan modulos 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import folium

#Se importa metodo de sklearn para la division de conjuntos de train y test del dataset
from sklearn.model_selection import train_test_split

#Se importan modelos de sklearn para entrenar el modelo y ver la diferencia entre Regresion Logistica y Decenso del Gradiente
from sklearn.linear_model import LogisticRegression, SGDClassifier

#Se importa el modelo de arbol de decision solo para evaluar caracteristicas importantes
from sklearn.tree import DecisionTreeClassifier

#Se importan metodos y herramientas de Sklearn para la parte de preprocesamiento
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder



<h1>INTRODUCCION</h1>

<p>En este trabajo se podra evidenciar el proceso al obtener un dataset, el cual constara de un EDA y asi mismo un arreglo de dichos datos, datos faltantes, outliers, datos categoricos, features mas importantes, etc. </p>
<p>Asi mismo se evaluara luego en el lenguaje de C++ para comparar el rendimiento y asi mismo la eficiencia de los modelos de ML, como es el caso de Regresion Logistica y Decenso del gradiente </p>

<b> El DataSet con el que se va a trabajar es un DataSet que contiene los delitos cometidos en la ciudad de Chicago entre el año 2012 al 2017 en la cual contiene solo delitos de 4 tipos que son:</b>

* THEFT
* CRIMINAL DAMAGE
* NARCOTICS
* ASSAULT
* PROSTITUTION. 

<p>En donde se puede evidenciar diferentes tipos de caracteristicas que tiene de informacion el dataset para definir si el caso finalizo con una sentencia o no</p>

<p>En el trabajo se realizara un analisis de los datos para poder determinar si se puede predecir si un caso finalizara con una sentencia o no, para esto se realizara un analisis de los datos para poder determinar que caracteristicas son las mas importantes, y arreglar el dataset original, esto con fin de tener un dataset mas acomodado al momento de realizar el proyecto de C++ y compararlo de manera mas sencilla</p>

# DATASET KAGGLE

In [ ]:
#Se extrae el dataset en una dataframe usando pandas
df =pd.read_csv('./Chicago_Crimes_2012_to_2017.csv')
#Se imprimen los 5 primeros registros del dataframe
df.head() 

## EDA

In [ ]:
#Se muestra la informacion de cada una de las columnas del dataframe para estar informado de que se le va a realizar mas adelante
#Como puede ser tipo de columna, valores nulos, cantidad de columnas, etc.
df.info()

In [ ]:
#Se imprimen los primeros 5 registros
df.head()

In [ ]:
#Se imprime la cantidad de registros y columnas que tiene el dataframe
df.shape

In [ ]:
#Se imprime los valores que contienen la columna 'Primary Type' el cual se refiere a los tipos de delitos que se encuentran
#en el dataset
df['Primary Type'].value_counts()

In [ ]:
#Se eliminan las columnas que no se van a utilizar manualmente. 
df.drop(['Block','Description','Location Description','Updated On','Location', 'Case Number', 'Date', 'IUCR','FBI Code', 'Unnamed: 0'], axis=1, inplace=True)

In [ ]:
#Se imprime los 2 primeros registros para evidenciar y asegurar que los cambios si se hayan registrado
df.head(2)

In [ ]:
#Se muestra la informacion de cada una de las columnas del dataframe para estar informado de que se le va a realizar mas adelante
df.info()

In [ ]:
#Se grafica la cantidad de crimenes por año con la libreria plotly
fig = px.histogram(df, x="Year", color="Year", marginal="box", hover_data=df.columns)
fig.show()

## LIMPIEZA, MANEJO Y PREPROCESAMIENTO DE DATASET

### IMPUTER

In [ ]:
#Se inicializa el imputador el cual trabajara con la estrategia de: Promedio
imputerNum = SimpleImputer(strategy='mean')
#Al evidenciar las columnas que contenian valores nulos se realiza el imputador en esas columnas
df[['District','Ward', 'Community Area', 'X Coordinate', 'Y Coordinate', 'Latitude', 'Longitude']] = imputerNum.fit_transform(df[['District','Ward', 'Community Area', 'X Coordinate', 'Y Coordinate', 'Latitude', 'Longitude']])


In [ ]:
#Se muestra la informacion de cada una de las columnas del dataframe donde se evidencia los cambios que se realizaron
df.info()

In [ ]:
#mapa de delitos con la columna de df "Latitude" y "Longitude"
mapa = folium.Map(location=[df.Latitude[8], df.Longitude[8]], zoom_start=13, tiles="Stamen Terrain")
for lat, lon in zip(df.Latitude[:10000], df.Longitude[:10000]):
    folium.CircleMarker([lat, lon], radius=4, color='red', fill=True, fill_color='red', fill_opacity=0.1).add_to(mapa)
mapa

<p> En este mapa didactico se evidencia en que lugar hubo dichos delitos mostrando la ubicacion y pudiendo interactuar con el mapa para ver la cantidad de delitos cometidos en cada lugar</p>

### PIPELINE Y TRANSFORMACION

In [ ]:
#Se realiza el pipeline y column transformer para convertir las columnas en especifico de categoria a numericos 

#pipeNum = Pipeline([('num', StandardScaler())])
pipeCat = Pipeline([('cat', OrdinalEncoder())])

#columnita = ColumnTransformer([('num', pipeNum, ['ID', 'Beat', 'District', 'Ward', 'Community Area', 'X Coordinate', 'Y Coordinate', 'Year','Latitude', 'Longitude']), ('cat', pipeCat, ['Primary Type', 'Arrest', 'Domestic'])])
columnita = ColumnTransformer([('cat', pipeCat, ['Primary Type', 'Arrest', 'Domestic'])])

In [ ]:
#Se imprime los primeros 5 registros para evidenciar los cambios mas adelante
df.head(5)

In [ ]:
#Se realiza la transformacion de las columnas categoricas
df[['Primary Type', 'Arrest', 'Domestic']] = columnita.fit_transform(df[['Primary Type', 'Arrest', 'Domestic']])
#Se imprime los primeros 5 registros donde se evidencian los cambios 
df.head(5)

In [ ]:
#Se reorganiza las columnas para dejar las columnas categoricas y el TARGET en la parte final
df = df[['ID', 'Beat', 'District', 'Ward', 'Community Area', 'X Coordinate', 'Y Coordinate', 'Year','Latitude', 'Longitude', 'Primary Type', 'Domestic', 'Arrest']]

In [ ]:
#Se imprime los primeros 2 registros para evidenciar los cambios registrados 
df.head(2)

In [ ]:
#Se muestra la informacion de cada una de las columnas del dataframe para evidenciar los cambios registrados
df.info()

## FEATURES IMPORTANTES

### DIVISION DE DATOS

<b> Una vez se arreglo el dataframe se realizara una forma de hallar los features importantes para poder realizar el modelo de ML, para esto se utilizara el metodo de Decision Tree Classifier 

In [ ]:
#Se divide los features y el target
X = df.drop('Arrest', axis=1)
y = df['Arrest']

In [ ]:
#Se evidencia los valores que contiene el Target
y.value_counts()

In [ ]:
#Se asegura de contener todos los features
X

In [ ]:
#Se divide el dataset en los conjuntos de entrenamiento y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Modelo para las caracteristicas importantes

In [ ]:
#crear el modelo de Arbol de decision para los features importantes
dt = DecisionTreeClassifier()
#se entrena el modelo
dt.fit(X_train, y_train)

In [ ]:
#Se realiza el score solo por curiosidad para evidenciar el procentaje al probar con los mismos datos en entrenamiento
dt.score(X_train, y_train)

In [ ]:
#Se realiza el score para evidenciar el procentaje al probar con los datos de test
#Y comprobar el aumento al escoger las caracteristicas importantes
dt.score(X_test, y_test)

<p><b> ANTES DE MOSTRAR LAS CARACTERISTICAS IMPORTANTES:</b> </p>
<p> LO ESPERADO POR EL AUTOR DEL CUADERNO AL MOSTRAR LOS FEATURES IMPORTANTES:</p>

1. La columna ID, debe tener muy bajo puntaje por ser una columna que no deberia ser importante al ser practicamente un numero aleatorio
2. La columna DOMESTIC, se espera que sea importante por ser una caracteristica que puede definir si el delito es o no domestico
3. Columnas como Latitude y Longitude o Coordenadas de X y Y, podria ser que no sea importante ya que simplemente son coordenadas de un lugar, aun que se puede tener en cuenta la localidad, o barrio de la ciudad de Chicago para saber si es un lugar peligroso o no
4. Por tal motivo se espera que la columna DISTRIT y COMMUNITY AREA, sean importantes ya que se puede tener en cuenta la localidad de la ciudad de Chicago para saber si es un lugar peligroso o no
5. La columna PRIMARY TYPE, se espera que sea importante ya que es el tipo de delito que se comete, y se puede tener en cuenta para saber si es un delito grave o no

### Se muestra las caracteristicas mas importantes

In [ ]:
#Primera visualizacion de los important features
dt.feature_importances_

In [ ]:
#Se visualiza el un grafico de barras para tenerlo mas claro
plt.figure(figsize=(12,8))
plt.barh(X.columns, dt.feature_importances_)
plt.show()

<h3> A través de los datos obtenidos se pudo evidenciar esto: </h3>


1. La columna ID fue considerada como el segundo feature mas importante del dataset, lo cual es un poco extraño ya que se esperaba que fuera la menos importante
2. La columna DOMESTIC y DISTRICT fue considerada como las caracteristicas menos importante del dataset, lo cual es un poco extraño ya que se esperaba que fueran de las mas importantes
3. La columna PRIMARY TYPE fue considerada como la caracteristica mas importante del dataset, lo cual es lo que se esperaba
4. Columnas como Coordenadas de X y Y, o Latitude y Longitude, fueron consideradas como las caracteristicas importantes debido a qe se tiene en cuenta si el barrio de la ciudad de Chicago es peligroso o no


In [ ]:
#Se realiza una prueba PERSONAL para verificar por que el feature Domestic no es tan importante
#para verificar si el feature_importances_ es correcto o no
X.iloc[60:100]

In [ ]:
y.iloc[60:100].values.reshape(1,-1)

<b> Al realizar la prueba se llega a la conclusion que aun que pueda ser una caracteristica importante, no fue asi debido a que la gran mayoria de los valores que contiene son de 1 valor, eso hace que aun que el TARGET vaya prediciendo diferentes valores, el modelo no pueda predecir bien debido a que la gran mayoria de los valores son de valor 0, por lo que se puede decir que la caracteristica no es importante al bajar dicha precision</b>

## Prueba de caracteristicas mas importantes

### PRUEBA #1: Se prueba con las caracteristicas mas importantes que se dijo en el modelo

In [ ]:
#Se elimina las columnas que no fueron importantes
X.drop(['Domestic','District','Community Area','Ward','Year'], axis=1, inplace=True)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train.head(2)

In [ ]:
#entrenar el modelo
dt.fit(X_train, y_train)

In [ ]:
#Se evalua el score del modelo nuevo con las caracteristicas importantes
dt.score(X_test, y_test)

<b> Se comprueba que el porcentaje de caracteristicas mas importantes es igual o incluso menor que el de la prueba anterior </b>
<p> Por lo que se realiza una eliminacion manual extra de caracteristicas </p>

### PRUEBA #2: Se realiza lo que se debio hacer desde un principio (Eliminar columna ID aun que hubiera sido de las más importantes)

In [ ]:
#Se elimina manualmente la columna ID
X.drop(['ID'], axis=1, inplace=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
dt.fit(X_train, y_train)

In [ ]:
#Se evalua el score del nuevo dataset de columnas mas importantes
dt.score(X_test, y_test)

<b> Se evidencia el incremento del porcentaje de la validacion con las nuevas caracteristicas, teniendo asi un mejor rendimiento en la prediccion </b>

## EXPORTACION DEL NUEVO DATASET

In [ ]:
#Se concatena los features y targets escogidos para dejar el dataset completo
df2 = pd.concat([X,y], axis=1)
df2.to_csv('dataSetArrestosParcial.csv', index=False)


# NUEVO DATASET

In [78]:
#Se extrae el nuevo DataSet para trabajar como si fuera a empezar de 0 con el nuevo dataset
dfParcial = pd.read_csv('./dataSetArrestosParcial.csv')
#Se imprime los primeros 5 registros para comprobar los datos
dfParcial.head()

,Beat,X Coordinate,Y Coordinate,Latitude,Longitude,Primary Type,Arrest
0,1523,1139890.0,1901675.0,41.886297,-87.761751,4.0,0.0
1,1232,1168776.0,1898793.0,41.877812,-87.655758,4.0,0.0
2,133,1179375.0,1886199.0,41.843017,-87.617227,4.0,0.0
3,1434,1160444.0,1910787.0,41.910901,-87.686019,4.0,0.0
4,1424,1162515.0,1909832.0,41.908237,-87.678437,4.0,0.0


## EDA

In [ ]:
#Se imprime la cantidad de registros y columnas que tiene el dataframe
dfParcial.shape

In [ ]:
#Se muestra la informacion de cada una de las columnas del dataframe para estar informado de que se le va a realizar mas adelante
#Como puede ser tipo de columna, valores nulos, cantidad de columnas, etc.
dfParcial.info()


In [ ]:
#Se emprime los primeros 2 registros para comprobar los datos
dfParcial.head(2)

In [ ]:
#Se imprime los valores que contienen la columna 'Primary Type' el cual se refiere a los tipos de delitos que se encuentran
#en el dataset
dfParcial['Primary Type'].value_counts()

In [ ]:
#Se grafica los diferentes tipos de delitos, comparando si hubo sentencia o no por delito
fig = px.histogram(dfParcial, x="Primary Type", color="Arrest", barmode="group", facet_col="Arrest", facet_col_wrap=2)
fig.show()

In [ ]:
#Para poder evidenciar dicha grafica de forma mas clara se realiza la comparacion de ambos
fig = px.histogram(dfParcial, x="Primary Type", color="Arrest", barmode="group")
fig.show()

In [ ]:
#mapa de delitos con la columna de df "Latitude" y "Longitude" para presenciar la distribucion de los crimenes
mapa = folium.Map(location=[df.Latitude[0], df.Longitude[0]], zoom_start=13)
for lat, lon in zip(df.Latitude[:4000], df.Longitude[:4000]):
    folium.CircleMarker([lat, lon], radius=4, color='red', fill=True, fill_color='red', fill_opacity=0.1).add_to(mapa)
mapa

## DIVISION DE DATOS

In [79]:
#Se divide el dataset en Target y Features
X = dfParcial.drop('Arrest', axis=1)
y = dfParcial['Arrest']
#Se separan en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
#Se muestran los datos para luego hacer cambios y poder comprobar dichos cambios
X_train

,Beat,X Coordinate,Y Coordinate,Latitude,Longitude,Primary Type
21119,1211,1157903.0,1907866.0,41.902938,-87.695433,2.0
222243,1434,1162771.0,1910740.0,41.910723,-87.677471,4.0
693884,1831,1175463.0,1903198.0,41.889752,-87.631073,4.0
406627,2431,1166984.0,1945439.0,42.005849,-87.660994,4.0
161685,331,1192054.0,1860145.0,41.771224,-87.571547,4.0
...,...,...,...,...,...,...
259178,1022,1155025.0,1893273.0,41.862951,-87.706396,0.0
365838,533,1183326.0,1817142.0,41.653426,-87.604874,0.0
131932,1332,1159476.0,1899930.0,41.881128,-87.689874,3.0
671155,331,1191523.0,1858333.0,41.766264,-87.573552,2.0


## LIMPIEZA, MANEJO Y PREPROCESAMIENTO DE DATASET

In [80]:
#Se crea el pipeline para Escalar los datos y entrenar el modelo
pipeNum = Pipeline([('num', StandardScaler())])

In [81]:
#Se transforman los datos Escalandolos con el pipeline
X_train.iloc[:,:-1] = pipeNum.fit_transform(X_train.iloc[:,:-1])

In [66]:
#Se comprueba los cambios realizados
X_train

,Beat,X Coordinate,Y Coordinate,Latitude,Longitude,Primary Type
21119,0.066844,-0.359049,0.631007,0.630686,-0.356013,2.0
222243,0.390283,-0.083012,0.718399,0.716691,-0.073623,4.0
693884,0.966093,0.636680,0.489064,0.485032,0.655844,4.0
406627,1.836335,0.155884,1.773514,1.767492,0.185426,4.0
161685,-1.209511,1.577463,-0.820076,-0.824280,1.591697,4.0
...,...,...,...,...,...,...
259178,-0.207283,-0.522244,0.187268,0.188979,-0.528372,0.0
365838,-0.916529,1.082547,-2.127696,-2.125525,1.067739,0.0
131932,0.242342,-0.269853,0.389692,0.389771,-0.268611,3.0
671155,-1.209511,1.547353,-0.875175,-0.879064,1.560173,2.0


## PROMEDIO Y DESVIACION ESTANDAR

In [ ]:
#Se usa el metodo describe para poder ver diferentes tipos de estadisticas de los datos.
dfParcial.describe()

In [ ]:
dfParcial.mean()

In [ ]:
dfParcial.std()

**Se puede presenciar como los resultados del cuaderno son exactamente igual a los resultados que se pueden evidenciar en C++**

## EVALUAR MODELOS

<h3><b>A continuacion se realiza los modelos de aprendizaje, en los cuales se realizara Regresion Logistica y Descenso del Gradiente
el cual se va a comparar los resultados con los presentados en C++ </b></h3>

### REGRESION LOGISTICA

In [82]:
#Se realiza el modelo de regresion Logistica para poder predecir si hubo sentencia o no
modeloLogistic = LogisticRegression()
#Se entrena el modelo
modeloLogistic.fit(X_train, y_train)

LogisticRegression()

In [83]:
##Se realiza el preprocesamiento de los datos del test para poder realizar la prediccion debido a que los datos con los que se entreno
##el modelo estaban escalados

###IMPORTANTE: Con los datos de test se transforman directamente, no se usa el metodo de fit_transform debido a que los datos de test
### no se deben ajustar por que se reemplazaria el preprocesamiento que se realizo con los datos de train, solo se deben de
### transformar para poder realizar la prediccion
X_test.iloc[:,:-1] = pipeNum.transform(X_test.iloc[:,:-1])

In [84]:
#Se realiza el score para evidenciar el porcentaje al probar con los datos de test
modeloLogistic.score(X_test, y_test)

0.7058394059075486

### SGD

In [104]:
#Se realiza el modelo de Descenso del Gradiente para poder predecir si hubo sentencia o no
modeloSGD = SGDClassifier(max_iter=1000, tol=0.01)
#Se entrena el modelo
modeloSGD.fit(X_train, y_train)


SGDClassifier(tol=0.01)

In [105]:
#Se realiza el score para evidenciar el porcentaje al probar con los datos de test
modeloSGD.score(X_test, y_test)

0.7082730711464649

<h2>Se puede evidenciar como el mejor modelo es el de Descenso del Grafiente, el cual tiene un porcentaje de 70.82%, un poco mejor que el de regresion Logistica, esto debido a que al usar este algoritmo de optimizacion permite disminuir el error que se puede evidenciar al predecir el target</h2>

# Conclusiones

* <h3>Se evidencia como la metrica Accuracy usando el metodo Score, nos da un mejor resultado al usar el algoritmo de optimizacion SGD
* <h3>Comparando los resultados de C++ y Python, se puede evidenciar como en C++ las metricas dio un resultado mucho mejor al momento de predecir, lo cual se intento replicar en Python
* <h3>Se evidencia como el algoritmo de optimizacion SGD, permite disminuir el error que se puede evidenciar al predecir el target
* <h3>El porcentaje de predicción no es demasiado alto, esto ya que los Features son coordenadas, direcciones, no es tan facil predecir correctamente como si fueran unas caracteristicas mas significativas como el color de los ojos, el color de la piel, etc.
* <h3>El modulo folium permite visualizar los datos en un mapa, lo cual es muy util para poder visualizar los datos de una manera mas facil y entendible
* <h3>Se pudo realizar un trabajo completo de ingeniero de datos, desde la extraccion de datos, limpieza, manejo, preprocesamiento, visualizacion, y esto atrajo el proceso de buscar diferentes maneras de poder realizar el mismo trabajo, lo cual fue muy interesante y enriquecedor.

# Referencias

* <h3>https://github.com/DanielOlarte-GitHub/DataSetsFamosos
* <h3>https://github.com/DanielOlarte-GitHub/MetricasDeRendimiento/tree/main/TERCERPARCIAL
* <h3>https://es.khanacademy.org/math/multivariable-calculus/applications-of-multivariable-derivatives/optimizing-multivariable-functions/a/what-is-gradient-descent#:~:text=El%20descenso%20de%20gradiente%20es%20un%20algoritmo%20que%20estima%20num%C3%A9ricamente,0%20como%20hemos%20visto%20antes.
* <h3>https://es.coursera.org/lecture/introduccion-al-aprendizaje-profundo/descenso-de-gradiente-aplicado-a-una-regresion-logistica-nM1Av